In [26]:
import numpy as np
import pandas as pd

In [27]:
pip install gensim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [28]:
train=pd.read_csv('../dataset/datagrand_2021_train.csv',sep=',')
train['text'] = train['text'].map(lambda a: a.replace('，', str(30355 - 1)))
train['text'] = train['text'].map(lambda a: a.replace('！', str(30355 - 2)))
train['text'] = train['text'].map(lambda a: a.split(" "))
train['text'] = train['text'].map(lambda a: [int(num) for num in a])
train['1-label'] = train['label'].map(lambda a: int(a.split('-')[0]))
train['2-label'] = train['label'].map(lambda a: int(a.split('-')[1]))

In [29]:
train['label'].nunique()

35

In [30]:
# 统计各类别的样本数
c = train.groupby(['label'],as_index=False)['label'].agg({'cnt':'count'})
c.sort_values("label",inplace=True)
c.describe()
rel_label = [0]*36
for line in c['label']:
    first,secod = line.split('-')
    rel_label[int(secod)] = int(first)
print(rel_label)
c

[0, 1, 2, 2, 1, 3, 2, 4, 6, 1, 1, 2, 5, 6, 2, 6, 7, 2, 8, 6, 6, 6, 5, 9, 5, 2, 10, 8, 6, 6, 5, 6, 6, 2, 6, 5]


,label,cnt
0,1-1,1092
1,1-10,622
2,1-4,799
3,1-9,1931
4,10-26,18
5,2-11,358
6,2-14,281
7,2-17,176
8,2-2,736
9,2-25,239


In [96]:
# 统计text的长度信息
train['len'] = train['text'].str.len()
train.describe()

,id,1-label,2-label,len
count,14009.000000,14009.000000,14009.000000,14009.000000
mean,7004.000000,2.993004,13.753159,44.805197
std,4044.194296,2.015951,10.990892,16.104233
min,0.000000,1.000000,1.000000,8.000000
25%,3502.000000,1.000000,4.000000,35.000000
50%,7004.000000,2.000000,9.000000,45.000000
75%,10506.000000,5.000000,25.000000,53.000000
max,14008.000000,10.000000,35.000000,334.000000


In [54]:
#使用正则匹配
import re
m = re.findall('\d+',',')
max_vob = 0
min_vob = 9999
max_length=0
for text in train['text']:
    vobs = text.split(' ')
    max_length=max(max_length,len(vobs))
    vobs  = [int(vob) for vob in vobs if len(re.findall('\d+', vob))>0]
    max_vob = max(max_vob, max(vobs))
    min_vob = min(min_vob, min(vobs))
print(max_vob,min_vob,max_length)

AttributeError: 'list' object has no attribute 'split'

In [55]:
test = pd.read_csv('../dataset/datagrand_2021_test.csv',sep=',')
test['text'] = test['text'].map(lambda a: a.replace('，', str(30355 - 1)))
test['text'] = test['text'].map(lambda a: a.replace('！', str(30355 - 2)))
test['text'] = test['text'].map(lambda a: a.split(" "))
test['text'] = test['text'].map(lambda a: [int(num) for num in a])
test['text'][0]

[12163,
 8224,
 13343,
 25257,
 3470,
 10636,
 1317,
 26183,
 10620,
 8194,
 13535,
 10449,
 10446,
 7800,
 929,
 17281,
 4397,
 9963,
 30354,
 22705,
 25427,
 15465,
 4054,
 11295,
 26336,
 27700,
 25585,
 13355,
 23349,
 26526,
 9601,
 5203,
 13317,
 4526,
 26564,
 14489,
 25843,
 25473,
 14950,
 26669,
 17281,
 7493,
 749,
 26823,
 23372,
 17281,
 25398,
 29627,
 11342]

In [66]:
# 训练Word Embedding
# skip-gram 对低频词有更好的训练效果
from gensim.models import Word2Vec
import torch
sent1 = [row for row in train['text']]
sent2 = [row for row in test['text']]
word2vec = Word2Vec(sent1+sent2,min_count=0,vector_size=300,workers=3, window =5, sg = 1, epochs=100)
word2vec.save('word2vec.model')

In [67]:
# 训练Word Embedding
# skip-gram 对低频词有更好的训练效果
model = Word2Vec.load('word2vec.model')
model.wv.vectors

array([[ 0.07497025,  0.1899006 ,  0.16560312, ...,  0.17079985,
         0.01429177, -0.05273504],
       [ 0.00182579,  0.17928721, -0.07195623, ..., -0.27504107,
         0.08361343,  0.04570998],
       [-0.02408699,  0.09146526,  0.02789167, ...,  0.03591527,
         0.19339553,  0.02397041],
       ...,
       [ 0.00063746,  0.08349318,  0.05978573, ..., -0.08336208,
         0.04402731,  0.02328173],
       [-0.02805742,  0.05952026,  0.05404408, ..., -0.03224092,
         0.03869519, -0.0217739 ],
       [ 0.05074402,  0.1106256 , -0.05272442, ..., -0.01594095,
        -0.0158499 ,  0.01630765]], dtype=float32)

In [89]:
pred_wordEmbedding = torch.tensor(model.wv.vectors)
print(pred_wordEmbedding.shape)
pred_wordEmbedding = torch.cat((torch.zeros(1,300),pred_wordEmbedding),dim=0)
pred_wordEmbedding

torch.Size([3455, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0750,  0.1899,  0.1656,  ...,  0.1708,  0.0143, -0.0527],
        [ 0.0018,  0.1793, -0.0720,  ..., -0.2750,  0.0836,  0.0457],
        ...,
        [ 0.0006,  0.0835,  0.0598,  ..., -0.0834,  0.0440,  0.0233],
        [-0.0281,  0.0595,  0.0540,  ..., -0.0322,  0.0387, -0.0218],
        [ 0.0507,  0.1106, -0.0527,  ..., -0.0159, -0.0158,  0.0163]])

In [56]:
import re
m = re.findall('\d+',',')
max_vob = 0
min_vob = 9999
max_length=0
for text in test['text']:
    vobs = text.split(' ')
    max_length=max(max_length,len(vobs))
    vobs  = [int(vob) for vob in vobs if len(re.findall('\d+', vob))>0]
    max_vob = max(max_vob, max(vobs))
    min_vob = min(min_vob, min(vobs))
print(max_vob,min_vob,max_length)

AttributeError: 'list' object has no attribute 'split'

In [9]:
import torch
a = torch.randn((4,5))
torch.max(a,1)

torch.return_types.max(
values=tensor([0.1448, 0.9003, 0.6007, 1.3291]),
indices=tensor([4, 1, 3, 0]))

In [10]:
submit = pd.read_csv('../dataset/sample_submission.csv',sep=',')
submit['label'][0]
mysubmit = pd.read_csv('../dataset/submit.csv',sep=',')
mysubmit['label'][0]

'2-6'

In [11]:
d = mysubmit.groupby(['label'],as_index=False)['label'].agg({'cnt':'count'})
d.sort_values("label",inplace=True)
d

,label,cnt
0,1-1,451
1,1-10,304
2,1-4,392
3,1-9,1013
4,2-11,33
5,2-14,65
6,2-2,401
7,2-25,113
8,2-3,448
9,2-33,282


In [12]:
from transformers import BertTokenizer
import torch
tokenizer = BertTokenizer(vocab_file='../dataset/vocab.txt')
inputs = tokenizer(
    "10988 8346 9955 27506 1845 29179 28438 10300 27321 9755 ， 27321 14873 29731 487 22191 10449 2617 1433 ， 24965 6527 26669 12567 17281 1506 979 13535 27321 8346",
    truncation=True, return_tensors='pt',max_length=512)
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101, 11098,  8456, 10065, 27616,  1955, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           118, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  101, 11098,  8456, 10065, 27616,  1955, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           118, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]])}

In [13]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = rand <0.15
mask_arr

tensor([[False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False]])

In [14]:
# 不mask cls 和 seq
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False]])

In [15]:
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

/home/yxb/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  """Entry point for launching an IPython kernel.


[5, 20]

In [16]:
inputs.input_ids[0, selection] = 103
inputs

{'input_ids': tensor([[  101, 11098,  8456, 10065, 27616,   103, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           103, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  101, 11098,  8456, 10065, 27616,  1955, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           118, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]])}